<a href="https://colab.research.google.com/github/naz-mul94/CS69002_9A_18CS60R63/blob/master/nazmul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Review Sentiment Analysis

In [0]:
import nltk

In [7]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Import Header files

In [0]:
import torch
import pandas as pd
import numpy as np
from string import punctuation
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

lemmatizer = WordNetLemmatizer()

In [0]:
url = 'https://github.com/binny-mathew/IITKGP_CS69002_Spring_2019/blob/master/Dataset/check.csv'
df1 = pd.read_csv(url, sep='\t')

In [11]:
from google.colab import files
uploaded = files.upload()

Saving Train_20K.csv to Train_20K.csv
Saving Test_5K.csv to Test_5K.csv


In [0]:
import io
df = pd.read_csv(open('Train_20K.csv', encoding='utf-8'), sep='\t')
df2 = pd.read_csv(open('Test_5K.csv', encoding='utf-8'), sep='\t')
#print (df2)

In [0]:
text_reviews_temp = df['text'].astype(str).tolist()

text_labels = df['label'].astype(int)

text_reviews = list()
#i = 1
sw = set(stopwords.words('english'))
for text_review in text_reviews_temp:
 # print (text_review)
  text_review = text_review.translate(str.maketrans('', '', punctuation))
  tokens = text_review.lower().split()
  #print (text_review)
  #print (tokens)
  #i=i+1
  
  
  terms = set(lemmatizer.lemmatize(term) for term in tokens)
  #print (terms)
  temp = terms.difference(sw)
  #print (temp)
  text_reviews.append(" ".join(temp))
  #print (text_reviews)
  #if i==2:
   # break;

In [0]:
test_reviews_temp = df2['text'].astype(str).tolist()
test_labels = df2['label'].astype(int)
test_reviews = list()
sw = set(stopwords.words('english'))
for test_review in test_reviews_temp:
  test_review = test_review.translate(str.maketrans('', '', punctuation))
  tokens = test_review.lower().split()
  terms = set(lemmatizer.lemmatize(term) for term in tokens)
  temp = terms.difference(sw)
  test_reviews.append(" ".join(temp))
  

In [0]:
def generate_word_ids(dataset):
  word_to_ix = {}
  count = {}
  #i=1
  for sent,_ in dataset:
      #if i==1:
       # i=i+1
        #print (sent)
      for word in sent.split():
          if word not in count:
            count[word] = 1
          else:
            count[word] = count[word] + 1
            if word not in word_to_ix:
              if count[word] >= 6:
                word_to_ix[word] = len(word_to_ix)
                
  return word_to_ix

In [0]:

data = tuple(zip(text_reviews,text_labels))

test_data = tuple(zip(test_reviews,test_labels))

In [17]:
word_to_ix = generate_word_ids(data+test_data)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2
print(VOCAB_SIZE)

22887


#TASK1

## Intro to PyTorch

In [0]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

**BE VERY CAREFUL WITH THE DIMENSIONS**

## Model Definition for the BOWClassifier

In [0]:
def make_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix))
    for word in sentence.split():
        if word in word_to_ix:
            vec[word_to_ix[word]]+=1
    return vec.view(1, -1)

def make_target(label):
    return torch.LongTensor([label]).cuda()


#TASK1A

In [0]:
class Task1A(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task1A, self).__init__()
    self.lin = nn.Linear(vocab_size, 50)
    self.lin1 = nn.Linear(50, num_labels)
    
  def forward(self, x):
    return F.softmax(self.lin1(self.lin(x)))

In [0]:
Task_1A = Task1A(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
# define a loss function and an optimizer
loss_functionTask1A = nn.NLLLoss()
optTask1A = torch.optim.SGD(Task_1A.parameters(), lr = 0.001)

In [23]:
# Task1A
# the training loop
for epoch in range(3):
    print(epoch)
    for instance, label in data:
        # get the training data
        Task_1A.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label))
        probs = Task_1A(bow_vec) # forward pass
        loss = loss_functionTask1A(probs, label)
        loss.backward()
        optTask1A.step()
    print('CURRENT LOSS: {}'.format(loss.data))
    
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

print('--- AFTER TRAINING ---')
predictions = list()
for instance, label in test_data:
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    logprobs = Task_1A(bow_vec.cuda())
    pred = np.argmax(logprobs.data.cpu().numpy())
    predictions.append(pred)
    
print('Precision: {}'.format(precision_score(test_labels,predictions)))
print('Recall: {}'.format(recall_score(test_labels,predictions)))
print('f1-score: {}'.format(f1_score(test_labels,predictions)))
print('Accuracy: {}'.format(accuracy_score(test_labels,predictions)))

0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


CURRENT LOSS: -0.7787696123123169
1
CURRENT LOSS: -0.9385720491409302
2
CURRENT LOSS: -0.9744940996170044
--- AFTER TRAINING ---
Precision: 0.8405742349829997
Recall: 0.8836378077839555
f1-score: 0.8615682478218781
Accuracy: 0.857


In [24]:
torch.save(Task_1A,'Task1A.mdl')

from google.colab import files
files.download("Task1A.mdl")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task1A. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#TASK1B

In [0]:
class Task1B(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task1B, self).__init__()
    self.lin = nn.Linear(vocab_size, 100)
    self.lin1 = nn.Linear(100, num_labels)
    
  def forward(self, x):
    return F.softmax(self.lin1(self.lin(x)))

In [0]:
Task_1B = Task1B(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
# define a loss function and an optimizer
loss_functionTask1B = nn.NLLLoss()
optTask1B = torch.optim.SGD(Task_1B.parameters(), lr = 0.001)

In [0]:
# Task1B
# the training loop
for epoch in range(3):
    print(epoch)
    for instance, label in data:
        # get the training data
        Task_1B.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label))
        probs = Task_1B(bow_vec) # forward pass
        loss = loss_functionTask1B(probs, label)
        loss.backward()
        optTask1B.step()
    print('CURRENT LOSS: {}'.format(loss.data))
    
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

print('--- AFTER TRAINING ---')
predictions = list()
for instance, label in test_data:
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    logprobs = Task_1B(bow_vec.cuda())
    pred = np.argmax(logprobs.data.cpu().numpy())
    predictions.append(pred)
    
print('Precision: {}'.format(precision_score(test_labels,predictions)))
print('Recall: {}'.format(recall_score(test_labels,predictions)))
print('f1-score: {}'.format(f1_score(test_labels,predictions)))
print('Accuracy: {}'.format(accuracy_score(test_labels,predictions)))

0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


CURRENT LOSS: -0.8110874891281128
1
CURRENT LOSS: -0.9488620758056641
2
CURRENT LOSS: -0.9782378673553467
--- AFTER TRAINING ---
Precision: 0.8456860509312049
Recall: 0.8836378077839555
f1-score: 0.8642454845601089
Accuracy: 0.8602


In [0]:
torch.save(Task_1B,'Task1B.mdl')

from google.colab import files
files.download("Task1B.mdl")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task1B. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#TASK1C

In [0]:
class Task1C(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task1C, self).__init__()
    self.lin = nn.Linear(vocab_size, 200)
    self.lin1 = nn.Linear(200, num_labels)
    
  def forward(self, x):
    return F.softmax(self.lin1(self.lin(x)))

In [0]:
Task_1C = Task1C(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
# define a loss function and an optimizer
loss_functionTask1C = nn.NLLLoss()
optTask1C = torch.optim.SGD(Task_1C.parameters(), lr = 0.005)

In [0]:
# Task1C
# the training loop
for epoch in range(3):
    print(epoch)
    for instance, label in data:
        # get the training data
        Task_1C.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label))
        probs = Task_1C(bow_vec) # forward pass
        loss = loss_functionTask1C(probs, label)
        loss.backward()
        optTask1C.step()
    print('CURRENT LOSS: {}'.format(loss.data))
    
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

print('--- AFTER TRAINING ---')
predictions = list()
for instance, label in test_data:
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    logprobs = Task_1C(bow_vec.cuda())
    pred = np.argmax(logprobs.data.cpu().numpy())
    predictions.append(pred)
    
print('Precision: {}'.format(precision_score(test_labels,predictions)))
print('Recall: {}'.format(recall_score(test_labels,predictions)))
print('f1-score: {}'.format(f1_score(test_labels,predictions)))
print('Accuracy: {}'.format(accuracy_score(test_labels,predictions)))

0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


CURRENT LOSS: -0.9914299845695496
1
CURRENT LOSS: -0.9968959093093872
2
CURRENT LOSS: -0.9987044334411621
--- AFTER TRAINING ---
Precision: 0.8769539078156313
Recall: 0.868943606036537
f1-score: 0.8729303810093758
Accuracy: 0.8726


In [0]:
torch.save(Task_1C,'Task1C.mdl')

from google.colab import files
files.download("Task1C.mdl")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task1C. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#TASK1D

In [0]:
class Task1D(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task1D, self).__init__()
    self.lin = nn.Linear(vocab_size, 500)
    self.lin1 = nn.Linear(500, num_labels)
    
  def forward(self, x):
    return F.softmax(self.lin1(self.lin(x)))

In [0]:
Task_1D = Task1D(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
# define a loss function and an optimizer
loss_functionTask1D = nn.NLLLoss()
optTask1D = torch.optim.SGD(Task_1D.parameters(), lr = 0.005)

In [0]:
# Task1D
# the training loop
for epoch in range(3):
    print(epoch)
    for instance, label in data:
        # get the training data
        Task_1D.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label))
        probs = Task_1D(bow_vec) # forward pass
        loss = loss_functionTask1D(probs, label)
        loss.backward()
        optTask1D.step()
    print('CURRENT LOSS: {}'.format(loss.data))
    
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

print('--- AFTER TRAINING ---')
predictions = list()
for instance, label in test_data:
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    logprobs = Task_1D(bow_vec.cuda())
    pred = np.argmax(logprobs.data.cpu().numpy())
    predictions.append(pred)
    
print('Precision: {}'.format(precision_score(test_labels,predictions)))
print('Recall: {}'.format(recall_score(test_labels,predictions)))
print('f1-score: {}'.format(f1_score(test_labels,predictions)))
print('Accuracy: {}'.format(accuracy_score(test_labels,predictions)))

0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


CURRENT LOSS: -0.9925312399864197
1
CURRENT LOSS: -0.9975941777229309
2
CURRENT LOSS: -0.9988064765930176
--- AFTER TRAINING ---
Precision: 0.8756497401039585
Recall: 0.8697378872120731
f1-score: 0.8726838015540945
Accuracy: 0.8722


In [0]:
torch.save(Task_1D,'Task1D.mdl')

from google.colab import files
files.download("Task1D.mdl")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task1D. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#TASK2A

In [0]:
class Task2A(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task2A, self).__init__()
    self.lin = nn.Linear(vocab_size, 10)
    self.lin1 = nn.Linear(10, 10)
    self.lin2 = nn.Linear(10, num_labels)
    
  def forward(self, x):
    return F.softmax(self.lin2(self.lin1(self.lin(x))))

In [0]:
Task_2A = Task2A(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
# define a loss function and an optimizer
loss_functionTask2A = nn.NLLLoss()
optTask2A = torch.optim.SGD(Task_2A.parameters(), lr = 0.006)

In [0]:
 #Task2A
# the training loop
for epoch in range(4):
    print(epoch)
    for instance, label in data:
        # get the training data
        Task_2A.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label))
        probs = Task_2A(bow_vec) # forward pass
        loss = loss_functionTask2A(probs, label)
        loss.backward()
        optTask2A.step()
    print('CURRENT LOSS: {}'.format(loss.data))
    
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

print('--- AFTER TRAINING ---')
predictions = list()
for instance, label in test_data:
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    logprobs = Task_2A(bow_vec.cuda())
    pred = np.argmax(logprobs.data.cpu().numpy())
    predictions.append(pred)
    
print('Precision: {}'.format(precision_score(test_labels,predictions)))
print('Recall: {}'.format(recall_score(test_labels,predictions)))
print('f1-score: {}'.format(f1_score(test_labels,predictions)))
print('Accuracy: {}'.format(accuracy_score(test_labels,predictions)))



0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


CURRENT LOSS: -0.9999973773956299
1
CURRENT LOSS: -0.9999998807907104
2
CURRENT LOSS: -0.9999998807907104
3
CURRENT LOSS: -1.0
--- AFTER TRAINING ---
Precision: 0.8631743549648163
Recall: 0.8768864177918984
f1-score: 0.8699763593380615
Accuracy: 0.868


In [0]:
torch.save(Task_2A,"Task2A.mdl")
from google.colab import files
files.download("Task2A.mdl")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task2A. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#Task2B

In [0]:
class Task2B(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task2B, self).__init__()
    self.lin = nn.Linear(vocab_size, 20)
    self.lin1 = nn.Linear(20, 10)
    self.lin2 = nn.Linear(10, num_labels)
    
  def forward(self, x):
    return F.softmax(self.lin2(self.lin1(self.lin(x))))

In [0]:
Task_2B = Task2B(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
# define a loss function and an optimizer
loss_functionTask2B = nn.NLLLoss()
optTask2B = torch.optim.SGD(Task_2B.parameters(), lr = 0.006)

In [0]:
 #Task2B
# the training loop
for epoch in range(4):
    print(epoch)
    for instance, label in data:
        # get the training data
        Task_2B.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label))
        probs = Task_2B(bow_vec) # forward pass
        loss = loss_functionTask2B(probs, label)
        loss.backward()
        optTask2B.step()
    print('CURRENT LOSS: {}'.format(loss.data))
    
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

print('--- AFTER TRAINING ---')
predictions = list()
for instance, label in test_data:
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    logprobs = Task_2B(bow_vec.cuda())
    pred = np.argmax(logprobs.data.cpu().numpy())
    predictions.append(pred)
    
print('Precision: {}'.format(precision_score(test_labels,predictions)))
print('Recall: {}'.format(recall_score(test_labels,predictions)))
print('f1-score: {}'.format(f1_score(test_labels,predictions)))
print('Accuracy: {}'.format(accuracy_score(test_labels,predictions)))



0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


CURRENT LOSS: -0.9998061060905457
1
CURRENT LOSS: -0.9999988079071045
2
CURRENT LOSS: -0.9982149600982666
3
CURRENT LOSS: -0.9999984502792358
--- AFTER TRAINING ---
Precision: 0.8704807310290028
Recall: 0.8701350277998412
f1-score: 0.8703078450844092
Accuracy: 0.8694


In [0]:
torch.save(Task_2B,"Task2B.mdl")
from google.colab import files
files.download("Task2B.mdl")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task2B. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#TASK2C

In [0]:
class Task2C(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task2C, self).__init__()
    self.lin = nn.Linear(vocab_size, 30)
    self.lin1 = nn.Linear(30, 30)
    self.lin2 = nn.Linear(30, num_labels)
    
  def forward(self, x):
    return F.softmax(self.lin2(self.lin1(self.lin(x))))

In [0]:
Task_2C = Task2C(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
# define a loss function and an optimizer
loss_functionTask2C = nn.NLLLoss()
optTask2C = torch.optim.SGD(Task_2C.parameters(), lr = 0.05)

In [0]:
 #Task2C
# the training loop
for epoch in range(4):
    print(epoch)
    for instance, label in data:
        # get the training data
        Task_2C.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label))
        probs = Task_2C(bow_vec) # forward pass
        loss = loss_functionTask2C(probs, label)
        loss.backward()
        optTask2C.step()
    print('CURRENT LOSS: {}'.format(loss.data))
    
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

print('--- AFTER TRAINING ---')
predictions = list()
for instance, label in test_data:
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    logprobs = Task_2C(bow_vec.cuda())
    pred = np.argmax(logprobs.data.cpu().numpy())
    predictions.append(pred)
    
print('Precision: {}'.format(precision_score(test_labels,predictions)))
print('Recall: {}'.format(recall_score(test_labels,predictions)))
print('f1-score: {}'.format(f1_score(test_labels,predictions)))
print('Accuracy: {}'.format(accuracy_score(test_labels,predictions)))



0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


CURRENT LOSS: -1.0
1
CURRENT LOSS: -1.0
2
CURRENT LOSS: -1.0
3
CURRENT LOSS: -1.0
--- AFTER TRAINING ---
Precision: 0.7370381868984788
Recall: 0.9428117553613979
f1-score: 0.8273218330719638
Accuracy: 0.8018


In [0]:
torch.save(Task_2C,"Task2C.mdl")
from google.colab import files
files.download("Task2C.mdl")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task2C. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#TASK2D

In [0]:
class Task2D(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task2D, self).__init__()
    self.lin = nn.Linear(vocab_size, 50)
    self.lin1 = nn.Linear(50, 50)
    self.lin2 = nn.Linear(50, num_labels)
    
  def forward(self, x):
    return F.softmax(self.lin2(self.lin1(self.lin(x))))

In [0]:
Task_2D = Task2D(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
# define a loss function and an optimizer
loss_functionTask2D = nn.NLLLoss()
optTask2D = torch.optim.SGD(Task_2D.parameters(), lr = 0.001)

In [0]:
 #Task2D
# the training loop
for epoch in range(4):
    print(epoch)
    for instance, label in data:
        # get the training data
        Task_2D.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label))
        probs = Task_2D(bow_vec) # forward pass
        loss = loss_functionTask2D(probs, label)
        loss.backward()
        optTask2D.step()
    print('CURRENT LOSS: {}'.format(loss.data))
    
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

print('--- AFTER TRAINING ---')
predictions = list()
for instance, label in test_data:
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    logprobs = Task_2D(bow_vec.cuda())
    pred = np.argmax(logprobs.data.cpu().numpy())
    predictions.append(pred)
    
print('Precision: {}'.format(precision_score(test_labels,predictions)))
print('Recall: {}'.format(recall_score(test_labels,predictions)))
print('f1-score: {}'.format(f1_score(test_labels,predictions)))
print('Accuracy: {}'.format(accuracy_score(test_labels,predictions)))



0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


CURRENT LOSS: -0.8653515577316284
1
CURRENT LOSS: -0.9867892861366272
2
CURRENT LOSS: -0.9961445331573486
3
CURRENT LOSS: -0.9964816570281982
--- AFTER TRAINING ---
Precision: 0.8597026305756768
Recall: 0.8955520254169976
f1-score: 0.8772612332231084
Accuracy: 0.8738


In [0]:
torch.save(Task_2D,"Task2D.mdl")
from google.colab import files
files.download("Task2D.mdl")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task2D. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#TASK2E

In [0]:
class Task2E(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task2E, self).__init__()
    self.lin = nn.Linear(vocab_size, 50)
    self.lin1 = nn.Linear(50, 100)
    self.lin2 = nn.Linear(100, num_labels)
    
  def forward(self, x):
    return F.softmax(self.lin2(self.lin1(self.lin(x))))

In [0]:
Task_2E = Task2E(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
# define a loss function and an optimizer
loss_functionTask2E = nn.NLLLoss()
optTask2E = torch.optim.SGD(Task_2E.parameters(), lr = 0.005)

In [41]:
 #Task2E
# the training loop
for epoch in range(4):
    print(epoch)
    for instance, label in data:
        # get the training data
        Task_2E.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label))
        probs = Task_2E(bow_vec) # forward pass
        loss = loss_functionTask2E(probs, label)
        loss.backward()
        optTask2E.step()
    print('CURRENT LOSS: {}'.format(loss.data))
    
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

print('--- AFTER TRAINING ---')
predictions = list()
for instance, label in test_data:
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    logprobs = Task_2E(bow_vec.cuda())
    pred = np.argmax(logprobs.data.cpu().numpy())
    predictions.append(pred)
    
print('Precision: {}'.format(precision_score(test_labels,predictions)))
print('Recall: {}'.format(recall_score(test_labels,predictions)))
print('f1-score: {}'.format(f1_score(test_labels,predictions)))
print('Accuracy: {}'.format(accuracy_score(test_labels,predictions)))



0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


CURRENT LOSS: -0.9999836683273315
1
CURRENT LOSS: -1.0
2
CURRENT LOSS: -1.0
3
CURRENT LOSS: -0.9999996423721313
--- AFTER TRAINING ---
Precision: 0.8717948717948718
Recall: 0.8641779189833201
f1-score: 0.8679696848823294
Accuracy: 0.8676


In [42]:
torch.save(Task_2E,"Task2E.mdl")
from google.colab import files
files.download("Task2E.mdl")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task2E. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#TASK3A

In [0]:
class Task3A(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task3A, self).__init__()
    self.lin = nn.Linear(vocab_size, 100)
    self.lin1 = nn.Linear(100, 50)
    self.lin2 = nn.Linear(50, 10)
    self.lin3 = nn.Linear(10, num_labels)
    
    
  def forward(self, x):
    return F.softmax(self.lin3(self.lin2(self.lin1(self.lin(x)))))

In [0]:
Task_3A = Task3A(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
# define a loss function and an optimizer
loss_functionTask3A = nn.NLLLoss()
optTask3A = torch.optim.SGD(Task_3A.parameters(), lr = 0.025)

In [35]:
# Task3A
# the training loop
for epoch in range(5):
    print(epoch)
    for instance, label in data:
        # get the training data
        Task_3A.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label))
        probs = Task_3A(bow_vec) # forward pass
        loss = loss_functionTask3A(probs, label)
        loss.backward()
        optTask3A.step()
    print('CURRENT LOSS: {}'.format(loss.data))
    
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

print('--- AFTER TRAINING ---')
predictions = list()
for instance, label in test_data:
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    logprobs = Task_3A(bow_vec.cuda())
    pred = np.argmax(logprobs.data.cpu().numpy())
    predictions.append(pred)
    
print('Precision: {}'.format(precision_score(test_labels,predictions)))
print('Recall: {}'.format(recall_score(test_labels,predictions)))
print('f1-score: {}'.format(f1_score(test_labels,predictions)))
print('Accuracy: {}'.format(accuracy_score(test_labels,predictions)))



0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


CURRENT LOSS: -1.0
1
CURRENT LOSS: -1.0
2
CURRENT LOSS: -1.0
3
CURRENT LOSS: -1.0
4
CURRENT LOSS: -1.0
--- AFTER TRAINING ---
Precision: 0.8565112540192926
Recall: 0.846306592533757
f1-score: 0.8513783459848183
Accuracy: 0.8512


In [36]:
torch.save(Task_3A,"Task3A.mdl")
from google.colab import files
files.download("Task3A.mdl")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task3A. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#TASK3B

In [0]:
class Task3B(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task3B, self).__init__()
    self.lin = nn.Linear(vocab_size, 200)
    self.lin1 = nn.Linear(200, 100)
    self.lin2 = nn.Linear(100, 10)
    self.lin3 = nn.Linear(10, num_labels)
    
    
  def forward(self, x):
    return F.softmax(self.lin3(self.lin2(self.lin1(self.lin(x)))))

In [0]:
Task_3B = Task3B(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
# define a loss function and an optimizer
loss_functionTask3B = nn.NLLLoss()
optTask3B = torch.optim.SGD(Task_3B.parameters(), lr = 0.005)

In [48]:
# Task3B
# the training loop
for epoch in range(5):
    print(epoch)
    for instance, label in data:
        # get the training data
        Task_3B.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label))
        probs = Task_3B(bow_vec) # forward pass
        loss = loss_functionTask3B(probs, label)
        loss.backward()
        optTask3B.step()
    print('CURRENT LOSS: {}'.format(loss.data))
    
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

print('--- AFTER TRAINING ---')
predictions = list()
for instance, label in test_data:
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    logprobs = Task_3B(bow_vec.cuda())
    pred = np.argmax(logprobs.data.cpu().numpy())
    predictions.append(pred)
    
print('Precision: {}'.format(precision_score(test_labels,predictions)))
print('Recall: {}'.format(recall_score(test_labels,predictions)))
print('f1-score: {}'.format(f1_score(test_labels,predictions)))
print('Accuracy: {}'.format(accuracy_score(test_labels,predictions)))


0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


CURRENT LOSS: -1.0
1
CURRENT LOSS: -1.0
2
CURRENT LOSS: -1.0
3
CURRENT LOSS: -0.9999985694885254
4
CURRENT LOSS: -1.0
--- AFTER TRAINING ---
Precision: 0.8659127625201939
Recall: 0.8514694201747418
f1-score: 0.8586303564277132
Accuracy: 0.8588


In [49]:
torch.save(Task_3B,"Task3B.mdl")
from google.colab import files
files.download("Task3B.mdl")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task3B. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#TASK4A

In [0]:
class Task4A(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task4A, self).__init__()
    self.lin = nn.Linear(vocab_size, 30)
    self.lin1 = nn.Linear(30, 20)
    self.lin2 = nn.Linear(20, 10)
    self.lin3 = nn.Linear(10, num_labels)
    
    self.droplayer = nn.Dropout(p=0.3)
    
    
  def forward(self, x):
    return F.softmax(self.lin3(self.droplayer(self.lin2(self.lin1(self.lin(x))))))

In [0]:
Task_4A = Task4A(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
# define a loss function and an optimizer
loss_functionTask4A = nn.NLLLoss()
optTask4A = torch.optim.SGD(Task_4A.parameters(), lr = 0.006)

In [53]:
# Task4A
# the training loop
for epoch in range(6):
    print(epoch)
    for instance, label in data:
        # get the training data
        Task_4A.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label))
        probs = Task_4A(bow_vec) # forward pass
        loss = loss_functionTask4A(probs, label)
        loss.backward()
        optTask4A.step()
    print('CURRENT LOSS: {}'.format(loss.data))
    
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

print('--- AFTER TRAINING ---')
predictions = list()
for instance, label in test_data:
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    logprobs = Task_4A(bow_vec.cuda())
    pred = np.argmax(logprobs.data.cpu().numpy())
    predictions.append(pred)
    
print('Precision: {}'.format(precision_score(test_labels,predictions)))
print('Recall: {}'.format(recall_score(test_labels,predictions)))
print('f1-score: {}'.format(f1_score(test_labels,predictions)))
print('Accuracy: {}'.format(accuracy_score(test_labels,predictions)))

0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


CURRENT LOSS: -0.9999998807907104
1
CURRENT LOSS: -1.0
2
CURRENT LOSS: -1.0
3
CURRENT LOSS: -1.0
4
CURRENT LOSS: -0.9423568248748779
5
CURRENT LOSS: -1.0
--- AFTER TRAINING ---
Precision: 0.8669093851132686
Recall: 0.8510722795869737
f1-score: 0.8589178356713427
Accuracy: 0.8592


In [54]:
torch.save(Task_4A,"Task4A.mdl")
from google.colab import files
files.download("Task4A.mdl")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task4A. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#TASK4B

In [0]:
class Task4B(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task4B, self).__init__()
    self.lin = nn.Linear(vocab_size, 100)
    self.lin1 = nn.Linear(100, 100)
    self.lin3 = nn.Linear(100, num_labels)
    
    self.droplayer = nn.Dropout(p=0.3)
    
    
  def forward(self, x):
    return F.softmax(self.lin3(self.droplayer(self.lin1(self.lin(x)))))

In [0]:
Task_4B = Task4B(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
# define a loss function and an optimizer
loss_functionTask4B = nn.NLLLoss()
optTask4B = torch.optim.SGD(Task_4B.parameters(), lr = 0.005)

In [58]:
# Task4B
# the training loop
for epoch in range(6):
    print(epoch)
    for instance, label in data:
        # get the training data
        Task_4B.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label))
        probs = Task_4B(bow_vec) # forward pass
        loss = loss_functionTask4B(probs, label)
        loss.backward()
        optTask4B.step()
    print('CURRENT LOSS: {}'.format(loss.data))
    
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

print('--- AFTER TRAINING ---')
predictions = list()
for instance, label in test_data:
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    logprobs = Task_4B(bow_vec.cuda())
    pred = np.argmax(logprobs.data.cpu().numpy())
    predictions.append(pred)
    
print('Precision: {}'.format(precision_score(test_labels,predictions)))
print('Recall: {}'.format(recall_score(test_labels,predictions)))
print('f1-score: {}'.format(f1_score(test_labels,predictions)))
print('Accuracy: {}'.format(accuracy_score(test_labels,predictions)))

0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


CURRENT LOSS: -0.9999246597290039
1
CURRENT LOSS: -0.9999189376831055
2
CURRENT LOSS: -0.9999996423721313
3
CURRENT LOSS: -0.9999901056289673
4
CURRENT LOSS: -1.0
5
CURRENT LOSS: -1.0
--- AFTER TRAINING ---
Precision: 0.8623924941360438
Recall: 0.8760921366163622
f1-score: 0.8691883372734436
Accuracy: 0.8672


In [59]:
torch.save(Task_4B,"Task4B.mdl")
from google.colab import files
files.download("Task4B.mdl")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task4B. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#TASK4C

In [0]:
class Task4C(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task4C, self).__init__()
    self.lin = nn.Linear(vocab_size, 100)
    self.lin1 = nn.Linear(100, 10)
    self.lin3 = nn.Linear(10, num_labels)
    
    self.droplayer = nn.Dropout(p=0.3)
    
    
  def forward(self, x):
    return F.softmax(self.lin3(self.droplayer(self.lin1(self.lin(x)))))

In [0]:
Task_4C = Task4C(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
# define a loss function and an optimizer
loss_functionTask4C = nn.NLLLoss()
optTask4C = torch.optim.SGD(Task_4C.parameters(), lr = 0.005)

In [63]:
# Task4C
# the training loop
for epoch in range(6):
    print(epoch)
    for instance, label in data:
        # get the training data
        Task_4C.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label))
        probs = Task_4C(bow_vec) # forward pass
        loss = loss_functionTask4C(probs, label)
        loss.backward()
        optTask4C.step()
    print('CURRENT LOSS: {}'.format(loss.data))
    
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

print('--- AFTER TRAINING ---')
predictions = list()
for instance, label in test_data:
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    logprobs = Task_4C(bow_vec.cuda())
    pred = np.argmax(logprobs.data.cpu().numpy())
    predictions.append(pred)
    
print('Precision: {}'.format(precision_score(test_labels,predictions)))
print('Recall: {}'.format(recall_score(test_labels,predictions)))
print('f1-score: {}'.format(f1_score(test_labels,predictions)))
print('Accuracy: {}'.format(accuracy_score(test_labels,predictions)))

0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


CURRENT LOSS: -0.9999735355377197
1
CURRENT LOSS: -0.9999998807907104
2
CURRENT LOSS: -1.0
3
CURRENT LOSS: -0.9999498128890991
4
CURRENT LOSS: -1.0
5
CURRENT LOSS: -1.0
--- AFTER TRAINING ---
Precision: 0.8619084561675717
Recall: 0.8824463860206513
f1-score: 0.8720565149136577
Accuracy: 0.8696


In [65]:
torch.save(Task_4C,"Task4C.mdl")
from google.colab import files
files.download("Task4C.mdl")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task4C. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#TASK5A

In [0]:
class Task5A(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task5A, self).__init__()
    self.lin = nn.Linear(vocab_size, 30)
    self.lin1 = nn.Linear(30, 20)
    self.lin2 = nn.Linear(20, 10)
    self.lin3 = nn.Linear(10, num_labels)
    
    self.droplayer = nn.Dropout(p=0.4)
    
    
  def forward(self, x):
    x=F.relu(self.lin(x))
    x=self.droplayer(x)
    x=F.relu(self.lin1(x))
    x=self.droplayer(x)
    x=F.relu(self.lin2(x))
    x=self.droplayer(x)
    x=F.softmax(self.lin3(x),dim=1)
    return x



In [0]:
Task_5A = Task5A(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
# define a loss function and an optimizer
loss_functionTask5A = nn.NLLLoss()
optTask5A = torch.optim.SGD(Task_5A.parameters(), lr = 0.003)

In [69]:
# Task5A
# the training loop
for epoch in range(6):
    print(epoch)
    for instance, label in data:
        # get the training data
        Task_5A.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label))
        probs = Task_5A(bow_vec) # forward pass
        loss = loss_functionTask5A(probs, label)
        loss.backward()
        optTask5A.step()
    print('CURRENT LOSS: {}'.format(loss.data))
    
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

print('--- AFTER TRAINING ---')
predictions = list()
for instance, label in test_data:
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    logprobs = Task_5A(bow_vec.cuda())
    pred = np.argmax(logprobs.data.cpu().numpy())
    predictions.append(pred)
    
print('Precision: {}'.format(precision_score(test_labels,predictions)))
print('Recall: {}'.format(recall_score(test_labels,predictions)))
print('f1-score: {}'.format(f1_score(test_labels,predictions)))
print('Accuracy: {}'.format(accuracy_score(test_labels,predictions)))


0
CURRENT LOSS: -0.9915962815284729
1
CURRENT LOSS: -0.9999995231628418
2
CURRENT LOSS: -1.0
3
CURRENT LOSS: -1.0
4
CURRENT LOSS: -1.0
5
CURRENT LOSS: -0.9998390674591064
--- AFTER TRAINING ---
Precision: 0.8296827794561934
Recall: 0.8725178713264495
f1-score: 0.8505613627564848
Accuracy: 0.8456


In [71]:
torch.save(Task_5A,"Task5A.mdl")
from google.colab import files
files.download("Task5A.mdl")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task5A. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#TASK5B

In [0]:
class Task5B(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task5B, self).__init__()
    self.lin = nn.Linear(vocab_size, 30)
    self.lin1 = nn.Linear(30, 20)
    self.lin2 = nn.Linear(20, 10)
    self.lin3 = nn.Linear(10, num_labels)
    
    self.droplayer = nn.Dropout(p=0.3)
    
    
  def forward(self, x):
    x=torch.tanh(self.lin(x))
    x=self.droplayer(x)
    x=torch.tanh(self.lin1(x))
    x=self.droplayer(x)
    x=torch.tanh(self.lin2(x))
    x=self.droplayer(x)
    x=F.softmax(self.lin3(x),dim=1)
    return x


In [0]:
Task_5B = Task5B(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
# define a loss function and an optimizer
loss_functionTask5B = nn.NLLLoss()
optTask5B = torch.optim.SGD(Task_5B.parameters(), lr = 0.004)

In [81]:
# Task5B
# the training loop
for epoch in range(5):
    print(epoch)
    for instance, label in data:
        # get the training data
        Task_5B.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label))
        probs = Task_5B(bow_vec) # forward pass
        loss = loss_functionTask5B(probs, label)
        loss.backward()
        optTask5B.step()
    print('CURRENT LOSS: {}'.format(loss.data))
    
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

print('--- AFTER TRAINING ---')
predictions = list()
for instance, label in test_data:
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    logprobs = Task_5B(bow_vec.cuda())
    pred = np.argmax(logprobs.data.cpu().numpy())
    predictions.append(pred)
    
print('Precision: {}'.format(precision_score(test_labels,predictions)))
print('Recall: {}'.format(recall_score(test_labels,predictions)))
print('f1-score: {}'.format(f1_score(test_labels,predictions)))
print('Accuracy: {}'.format(accuracy_score(test_labels,predictions)))

0
CURRENT LOSS: -0.999196469783783
1
CURRENT LOSS: -0.9999500513076782
2
CURRENT LOSS: -0.9999939203262329
3
CURRENT LOSS: -0.9999774694442749
4
CURRENT LOSS: -1.0
--- AFTER TRAINING ---
Precision: 0.8876843018213356
Recall: 0.812946783161239
f1-score: 0.8486733001658374
Accuracy: 0.854


In [82]:
torch.save(Task_5B,"Task5B.mdl")
from google.colab import files
files.download("Task5B.mdl")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task5B. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#TASK5C

In [0]:
class Task5C(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task5C, self).__init__()
    self.lin = nn.Linear(vocab_size, 30)
    self.lin1 = nn.Linear(30, 20)
    self.lin2 = nn.Linear(20, 10)
    self.lin3 = nn.Linear(10, num_labels)
    
    self.droplayer = nn.Dropout(p=0.2)
    
    
  def forward(self, x):
    x=torch.sigmoid(self.lin(x))
    x=self.droplayer(x)
    x=torch.sigmoid(self.lin1(x))
    x=self.droplayer(x)
    x=torch.sigmoid(self.lin2(x))
    x=self.droplayer(x)
    x=F.softmax(self.lin3(x),dim=1)
    return x

In [0]:
Task_5C = Task5C(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
# define a loss function and an optimizer
loss_functionTask5C = nn.NLLLoss()
optTask5C = torch.optim.SGD(Task_5C.parameters(), lr = 0.0005)

In [102]:
# Task5C
# the training loop
for epoch in range(6):
    print(epoch)
    for instance, label in data:
        # get the training data
        Task_5C.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label))
        probs = Task_5C(bow_vec) # forward pass
        loss = loss_functionTask5C(probs, label)
        loss.backward()
        optTask5C.step()
    print('CURRENT LOSS: {}'.format(loss.data))
    
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

print('--- AFTER TRAINING ---')
predictions = list()
for instance, label in test_data:
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    logprobs = Task_5C(bow_vec.cuda())
    pred = np.argmax(logprobs.data.cpu().numpy())
    predictions.append(pred)
    
print('Precision: {}'.format(precision_score(test_labels,predictions)))
print('Recall: {}'.format(recall_score(test_labels,predictions)))
print('f1-score: {}'.format(f1_score(test_labels,predictions)))
print('Accuracy: {}'.format(accuracy_score(test_labels,predictions)))

0
CURRENT LOSS: -0.434597909450531
1
CURRENT LOSS: -0.5622624158859253
2
CURRENT LOSS: -0.4247850179672241
3
CURRENT LOSS: -0.4831489622592926
4
CURRENT LOSS: -0.4363243877887726
5
CURRENT LOSS: -0.43503278493881226
--- AFTER TRAINING ---
Precision: 0.5025114653854553
Recall: 0.9138204924543288
f1-score: 0.6484430040862336
Accuracy: 0.501


In [104]:
torch.save(Task_5C,"Task5C.mdl")
from google.colab import files
files.download("Task5C.mdl")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task5C. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


##Generate the BOW Vectors

In [0]:
# define a loss function and an optimizer
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.025)